# 【問題1】SimpleRNNのフォワードプロパゲーション実装
SimpleRNNのクラスSimpleRNNを作成してください。基本構造はFCクラスと同じになります。

今回はバッチサイズをbatch_size、入力の特徴量数をn_features、RNNのノード数をn_nodesとして表記します。  
活性化関数はtanhとして進めますが、これまでのニューラルネットワーク同様にReLUなどに置き換えられます。

フォワードプロパゲーションの数式は以下のようになります。ndarrayのshapeがどうなるかを併記しています。

<center>$a_t = x_{t}\cdot W_{x} + h_{t-1}\cdot W_{h} + b\\
h_t = tanh(a_t)$</center>


a
t
 : 時刻tの活性化関数を通す前の状態 (batch_size, n_nodes)

h
t
 : 時刻tの状態・出力 (batch_size, n_nodes)

x
t
 : 時刻tの入力 (batch_size, n_features)

W
x
 : 入力に対する重み (n_features, n_nodes)

h
t
−
1
 : 時刻t-1の状態（前の時刻から伝わる順伝播） (batch_size, n_nodes)

W
h
 : 状態に対する重み。 (n_nodes, n_nodes)

b
 : バイアス項 (1,)
初期状態$h0$は全て0とすることが多いですが、任意の値を与えることも可能です。

上記の処理を系列数n_sequences回繰り返すことになります。

RNN全体への入力xは(batch_size, n_sequences, n_features)のような配列で渡されることになり、そこから各時刻の配列を取り出していきます。

分類問題であれば、それぞれの時刻のhに対して全結合層とソフトマックス関数（またはシグモイド関数）を使用します。

出力は最後のhだけを使用する場合と、全てのhを使う場合があります。

In [5]:
import numpy as np

In [18]:
class SimpleRNN:
    def __init__(self, Wx, Wh, b):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)] #パラメータ初期値設定
        self.cache = None
        
    def forward(self, x, h_prev):
        Wx, Wh, b = self.params
        for i in range(x.shape[1]):
            at = np.dot(x[:,i,:], Wx) + np.dot(h_prev, Wh) + b #フォワードプロパゲーションを行う
            h_next = np.tanh(at) #活性化関数をかませる
            h_prev = h_next #直前のh_nextをh_prevに代入
        self.cache = (x, h_prev, h_next)
        return h_next

# 【問題2】小さな配列でのフォワードプロパゲーションの実験
小さな配列でフォワードプロパゲーションを考えてみます。

入力x、初期状態h、重みw_xとw_h、バイアスbを次のようにします。

ここで配列xの軸はバッチサイズ、系列数、特徴量数の順番です。

＜検算用数値＞

In [13]:
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100
batch_size = x.shape[0] # 1
n_sequences = x.shape[1] # 3
n_features = x.shape[2] # 2
n_nodes = w_x.shape[1] # 4
h = np.zeros((batch_size, n_nodes))
b = np.array([1])

＜検算実施＞

In [14]:
rnn = SimpleRNN(w_x,w_h,b)
h_test = rnn.forward(x,h)
h_test

array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]])

＜正解＞

In [15]:
h_test_answer = np.array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]])

# 【問題3】（アドバンス課題）バックプロパゲーションの実装
バックプロパゲーションを実装します。

RNNの内部は全結合層を組み合わせた形になっているので、更新式は全結合層などと同様です。
<center>$W_x^{\prime} = W_x - \alpha E(\frac{\partial L}{\partial W_x}) \\
W_h^{\prime} = W_h - \alpha E(\frac{\partial L}{\partial W_h}) \\
b^{\prime} = b - \alpha E(\frac{\partial L}{\partial b})$</center>

＜以下未完成＞

In [20]:
class SimpleRNN_advanced:
    def __init__(self, Wx, Wh, b):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)] #パラメータ初期値設定
        self.cache = None
        
    def forward(self, x, h_prev):
        Wx, Wh, b = self.params
        for i in range(x.shape[1]):
            at = np.dot(x[:,i,:], Wx) + np.dot(h_prev, Wh) + b #フォワードプロパゲーションを行う
            h_next = np.tanh(at) #活性化関数をかませる
            h_prev = h_next #直前のh_nextをh_prevに代入
        self.cache = (x, h_prev, h_next)
        return h_next
    
    def backward(self, dh_next):
        Wx, Wh, b = self.params
        x, h_prev, h_next = self.cache
        dt = dh_next * (1 - h_next ** 2)
        db = np.sum(dt, axis=0)
        dWh = np.dot(h_prev.T, dt)
        dh_prev = np.dot(dt, Wh.T)
        dWx = np.dot(x.T, dt)
        dx = np.dot(dt, Wx.T)
        self.grads[0][...] = dWx
        self.grads[1][...] = dWh
        self.grads[2][...] = db
        return dx, dh_prev

In [30]:
rnn_advanced = SimpleRNN_advanced(w_x,w_h,b)

In [31]:
rnn_advanced.forward(x,h)
rnn_advanced.backward(h_test)

ValueError: could not broadcast input array from shape (2,3,4) into shape (2,4)